### Importing libraries

In [1]:
import pypots
import os
import sys
from pypots.utils.metrics import calc_mae
from pypots.optim import Adam
from pypots.imputation import SAITS, BRITS
import numpy as np
import benchpots
from pypots.utils.random import set_random_seed
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd    

2025-02-03 21:49:26.376202: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-03 21:49:26.391116: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-03 21:49:26.395744: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-03 21:49:26.406866: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/reformer/local_attention.py:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)
/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/reformer/local_attention.py:98: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)


### Loading database

In [2]:
set_random_seed()

from pypotsModify.benchpotsMAE.datasets import preprocess_physionet2012

# Load the PhysioNet-2012 dataset
physionet2012_dataset = preprocess_physionet2012(subset="all", rate=0.1)

# Take a look at the generated PhysioNet-2012 dataset, you'll find that everything has been prepared for you,
# data splitting, normalization, additional artificially-missing values for evaluation, etc.
print(physionet2012_dataset.keys())

2025-02-03 21:49:29 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-02-03 21:49:29 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-02-03 21:49:29 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-02-03 21:49:29 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-02-03 21:49:29 [INFO]: Loaded successfully!
2025-02-03 21:49:43 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-02-03 21:49:43 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-02-03 21:49:43 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

In [3]:
# assemble the datasets for training
dataset_for_training = {
    "X": physionet2012_dataset['train_X'],
}
# assemble the datasets for validation
dataset_for_validating = {
    "X": physionet2012_dataset['val_X'],
    "X_ori": physionet2012_dataset['val_X_ori'],
}

dataset_for_testing_ori = {
    "X_ori": physionet2012_dataset['test_X_ori'],
    "female_gender_test_X_ori": physionet2012_dataset['female_gender_test_X_ori'],
    "male_gender_test_X_ori": physionet2012_dataset['male_gender_test_X_ori'],
    "undefined_gender_test_X_ori": physionet2012_dataset['undefined_gender_test_X_ori'],
    "more_than_or_equal_to_65_test_X_ori":  physionet2012_dataset['more_than_or_equal_to_65_test_X_ori'],
    "less_than_65_test_X_ori": physionet2012_dataset['less_than_65_test_X_ori'],
    "ICUType_1_test_X_ori": physionet2012_dataset['ICUType_1_test_X_ori'],
    "ICUType_2_test_X_ori": physionet2012_dataset['ICUType_2_test_X_ori'],
    "ICUType_3_test_X_ori": physionet2012_dataset['ICUType_3_test_X_ori'],
    "ICUType_4_test_X_ori": physionet2012_dataset['ICUType_4_test_X_ori'],
    "classificacao_undefined_test_X_ori": physionet2012_dataset['classificacao_undefined_test_X_ori'],
    "classificacao_baixo_peso_test_X_ori": physionet2012_dataset['classificacao_baixo_peso_test_X_ori'],
    "classificacao_normal_peso_test_X_ori": physionet2012_dataset['classificacao_normal_peso_test_X_ori'],
    "classificacao_sobrepeso_test_X_ori": physionet2012_dataset['classificacao_sobrepeso_test_X_ori'],
    "classificacao_obesidade_1_test_X_ori": physionet2012_dataset['classificacao_obesidade_1_test_X_ori'],
    "classificacao_obesidade_2_test_X_ori": physionet2012_dataset['classificacao_obesidade_2_test_X_ori'],
    "classificacao_obesidade_3_test_X_ori": physionet2012_dataset['classificacao_obesidade_3_test_X_ori']
}

# assemble the datasets for test
dataset_for_testing = {
    "X": physionet2012_dataset['test_X'],
    "female_gender_test_X": physionet2012_dataset['female_gender_test_X'],
    "male_gender_test_X": physionet2012_dataset['male_gender_test_X'],
    "undefined_gender_test_X": physionet2012_dataset['undefined_gender_test_X'],
    "more_than_or_equal_to_65_test_X":  physionet2012_dataset['more_than_or_equal_to_65_test_X'],
    "less_than_65_test_X": physionet2012_dataset['less_than_65_test_X'],
    "ICUType_1_test_X": physionet2012_dataset['ICUType_1_test_X'],
    "ICUType_2_test_X": physionet2012_dataset['ICUType_2_test_X'],
    "ICUType_3_test_X": physionet2012_dataset['ICUType_3_test_X'],
    "ICUType_4_test_X": physionet2012_dataset['ICUType_4_test_X'],
    "classificacao_undefined_test_X": physionet2012_dataset['classificacao_undefined_test_X'],
    "classificacao_baixo_peso_test_X": physionet2012_dataset['classificacao_baixo_peso_test_X'],
    "classificacao_normal_peso_test_X": physionet2012_dataset['classificacao_normal_peso_test_X'],
    "classificacao_sobrepeso_test_X": physionet2012_dataset['classificacao_sobrepeso_test_X'],
    "classificacao_obesidade_1_test_X": physionet2012_dataset['classificacao_obesidade_1_test_X'],
    "classificacao_obesidade_2_test_X": physionet2012_dataset['classificacao_obesidade_2_test_X'],
    "classificacao_obesidade_3_test_X": physionet2012_dataset['classificacao_obesidade_3_test_X']
}

## calculate the mask to indicate the ground truth positions in test_X_ori, will be used by metric funcs to evaluate models
test_X_indicating_mask = []
test_X_ori = []
for i, j in zip(dataset_for_testing_ori.values(), dataset_for_testing.values()):
    test_X_indicating_mask.append(np.isnan(i) ^ np.isnan(j))
    test_X_ori.append(np.nan_to_num(i))# metric functions do not accpet input with NaNs, hence fill NaNs with 0


teste = "Teste"


In [4]:
test_X_indicating_mask_variable = []
test_X_ori_variable = []
for i in range(len(test_X_indicating_mask)):
    test_X_indicating_mask_variable.append(test_X_indicating_mask[i].reshape(37, len(test_X_indicating_mask[i]) * 48))
    test_X_ori_variable.append(test_X_ori[i].reshape(37, len(test_X_ori[i]) * 48))

### Inicialize the models

In [5]:
saits = SAITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/saits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-03 21:49:52 [INFO]: No given device, using default device: cuda
2025-02-03 21:49:52 [INFO]: Model files will be saved to tutorial_results/imputation/saits/20250203_T214952
2025-02-03 21:49:52 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/saits/20250203_T214952/tensorboard
2025-02-03 21:49:52 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


In [44]:
brits = BRITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/brits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-03 21:14:13 [INFO]: No given device, using default device: cuda
2025-02-03 21:14:13 [INFO]: Model files will be saved to tutorial_results/imputation/brits/20250203_T211413
2025-02-03 21:14:13 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/brits/20250203_T211413/tensorboard
2025-02-03 21:14:13 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


### Train the models

In [6]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
saits.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-02-03 21:50:03 [INFO]: Epoch 001 - training loss: 0.7230, validation loss: 6.7596
2025-02-03 21:50:06 [INFO]: Epoch 002 - training loss: 0.5370, validation loss: 6.7420
2025-02-03 21:50:09 [INFO]: Epoch 003 - training loss: 0.4953, validation loss: 6.7242
2025-02-03 21:50:11 [INFO]: Epoch 004 - training loss: 0.4641, validation loss: 6.7040
2025-02-03 21:50:14 [INFO]: Epoch 005 - training loss: 0.4387, validation loss: 6.6843
2025-02-03 21:50:17 [INFO]: Epoch 006 - training loss: 0.4206, validation loss: 6.6695
2025-02-03 21:50:20 [INFO]: Epoch 007 - training loss: 0.4065, validation loss: 6.6588
2025-02-03 21:50:22 [INFO]: Epoch 008 - training loss: 0.3963, validation loss: 6.6534
2025-02-03 21:50:25 [INFO]: Epoch 009 - training loss: 0.3867, validation loss: 6.6520
2025-02-03 21:50:28 [INFO]: Epoch 010 - training loss: 0.3793, validation loss: 6.6574
2025-02-03 21:50:28 [INFO]: Finished training. The best model is from epoch#9.
2025-02-03 21:50:28 [INFO]: Saved the model to tuto

In [ ]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
brits.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-02-03 20:42:18 [INFO]: Epoch 001 - training loss: 0.9367, validation loss: 6.7860
2025-02-03 20:43:06 [INFO]: Epoch 002 - training loss: 0.7334, validation loss: 6.7463
2025-02-03 20:43:55 [INFO]: Epoch 003 - training loss: 0.6826, validation loss: 6.7330
2025-02-03 20:44:44 [INFO]: Epoch 004 - training loss: 0.6580, validation loss: 6.7299
2025-02-03 20:45:34 [INFO]: Epoch 005 - training loss: 0.6434, validation loss: 6.7283
2025-02-03 20:46:23 [INFO]: Epoch 006 - training loss: 0.6319, validation loss: 6.7274
2025-02-03 20:47:13 [INFO]: Epoch 007 - training loss: 0.6230, validation loss: 6.7291
2025-02-03 20:48:02 [INFO]: Epoch 008 - training loss: 0.6156, validation loss: 6.7310
2025-02-03 20:48:51 [INFO]: Epoch 009 - training loss: 0.6090, validation loss: 6.7335
2025-02-03 20:48:51 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-02-03 20:48:51 [INFO]: Finished training. The best model is from epoch#6.
2025-02-03 20:48:51 [INFO]: Saved the mo

### The testing stage

In [7]:
# the testing stage, impute the originally-missing values and artificially-missing values in the test set
saits_imputation = []
for value in  dataset_for_testing.values():
   _dict = {'X':value}
   saits_results = saits.predict(_dict)
   saits_imputation.append(saits_results["imputation"])  

teste = 'Teste'


In [8]:
saits_imputation_variable = []
for i in range(len(saits_imputation)):
    saits_imputation_variable.append(saits_imputation[i].reshape(37, len(saits_imputation[i]) * 48))

teste = 'Teste'


In [13]:
# the testing stage, impute the originally-missing values and artificially-missing values in the test set
brits_imputation = []
for value in dataset_for_testing.values():
    _dict = {'X':value}
    brits_results = brits.predict(_dict)
    brits_imputation.append(brits_results["imputation"])

In [14]:
brits_imputation_variable = []
for i in range(len(brits_imputation)):
    brits_imputation_variable.append(brits_imputation[i].reshape(37, len(brits_imputation[i]) * 48))

### Calculate mean absolute error

In [9]:
testing_mae_saits_append_subgroups = []
testing_mae_saits_append_variables = []
for i in range(len(saits_imputation_variable)):
    for j in range(len(saits_imputation_variable[i])):
        testing_mae_saits_append_variables.append(calc_mae(saits_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_saits_append_subgroups.append(testing_mae_saits_append_variables)
    testing_mae_saits_append_variables = []
Teste = 'Teste'  

In [16]:
len(saits_imputation_variable[0][0])

115152

In [17]:
saits_imputation_variable[0][0]

array([-0.17572749, -0.33525062, -0.2920901 , ..., -0.39829695,
       -0.4246574 , -0.32629728], dtype=float32)

In [18]:
testing_mae_brits_append_subgroups = []
testing_mae_brits_append_variables = []
for i in range(len(brits_imputation_variable)):
    for j in range(len(brits_imputation_variable[i])):
      testing_mae_brits_append_variables.append(calc_mae(brits_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_brits_append_subgroups.append(testing_mae_brits_append_variables)
    testing_mae_brits_append_variables = []
Teste = 'Teste'      

In [10]:
subgroups = ["General", "Female", "Male", "Undefined Gender", "+65", "-65", "ICUType 1", "ICUType 2", "ICUType 3", "ICUType 4", "Undefined classification", "Low Weight", "Normal Weight", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"]

In [12]:
variables = ["ALP", "ALT", "AST", "Albumin", "BUN", "Bilirubin", "Cholesterol", "Creatinine", "DiasABP", "FiO2", "GCS", "Glucose", "HCO3", "HCT", "HR", "K", "Lactate", "MAP", "MechVent", "Mg", "NIDiasABP", "NIMAP", "NISysABP", "Na", "PaCO2", "PaO2", "Platelets", "RespRate", "SaO2", "SysABP", "Temp", "TroponinI", "TroponinT", "Urine", "WBC", "Weight", "Ph"]

In [13]:
print("SAITS - MAE")
print("************")
for i in range(len(subgroups)): 
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_saits_append_subgroups[i][j])

SAITS - MAE
************
General
-------------
ALP : 0.2460293682153394
ALT : 0.22043134772111558
AST : 0.26230819338030253
Albumin : 0.30293279432474296
BUN : 0.23659828366631006
Bilirubin : 0.2312914200797686
Cholesterol : 0.251679622012055
Creatinine : 0.2557353119481593
DiasABP : 0.23659807986738537
FiO2 : 0.2439170004762481
GCS : 0.27224084306168234
Glucose : 0.26067426800470134
HCO3 : 0.25306096736987144
HCT : 0.23665193821730493
HR : 0.22776729540049326
K : 0.23719489152605258
Lactate : 0.2550807744721406
MAP : 0.2596030041903261
MechVent : 0.2590311293507116
Mg : 0.2570448834883059
NIDiasABP : 0.2297923796966098
NIMAP : 0.23891888162436045
NISysABP : 0.25352442673224646
Na : 0.2576979141754115
PaCO2 : 0.22809380110830693
PaO2 : 0.2380665948777946
Platelets : 0.23528387027030165
RespRate : 0.25253001590961144
SaO2 : 0.255004959199687
SysABP : 0.2352274283737271
Temp : 0.24351372214867664
TroponinI : 0.24266481480201985
TroponinT : 0.2294270462157759
Urine : 0.24475188600873804
W

In [26]:
testing_mae_saits_append_subgroups

[[0.2460293682153394,
  0.22043134772111558,
  0.26230819338030253,
  0.30293279432474296,
  0.23659828366631006,
  0.2312914200797686,
  0.251679622012055,
  0.2557353119481593,
  0.23659807986738537,
  0.2439170004762481,
  0.27224084306168234,
  0.26067426800470134,
  0.25306096736987144,
  0.23665193821730493,
  0.22776729540049326,
  0.23719489152605258,
  0.2550807744721406,
  0.2596030041903261,
  0.2590311293507116,
  0.2570448834883059,
  0.2297923796966098,
  0.23891888162436045,
  0.25352442673224646,
  0.2576979141754115,
  0.22809380110830693,
  0.2380665948777946,
  0.23528387027030165,
  0.25253001590961144,
  0.255004959199687,
  0.2352274283737271,
  0.24351372214867664,
  0.24266481480201985,
  0.2294270462157759,
  0.24475188600873804,
  0.22730972748195327,
  0.23935864822125816,
  0.23457901976669757],
 [0.22045180764341177,
  0.23144723497907807,
  0.2633945857683271,
  0.24563074075348695,
  0.22405321283976448,
  0.25056365177769424,
  0.23170524298743803,
  0.2

In [25]:
df_saits_mae = pd.DataFrame(variables)
df_saits_mae

for i in range(len(variables)):
   for j in range(len(subgroups)):
        df_saits_mae[subgroups[j]] = testing_mae_saits_append_subgroups[j][i]


In [23]:
df_saits_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.234579,0.234759,0.275538,0.0,0.243482,0.279206,0.214579,0.193529,0.286102,0.244434,0.289764,0.478818,0.239752,0.239312,0.229589,0.243182,0.278581
1,ALT,0.234579,0.234759,0.275538,0.0,0.243482,0.279206,0.214579,0.193529,0.286102,0.244434,0.289764,0.478818,0.239752,0.239312,0.229589,0.243182,0.278581
2,AST,0.234579,0.234759,0.275538,0.0,0.243482,0.279206,0.214579,0.193529,0.286102,0.244434,0.289764,0.478818,0.239752,0.239312,0.229589,0.243182,0.278581
3,Albumin,0.234579,0.234759,0.275538,0.0,0.243482,0.279206,0.214579,0.193529,0.286102,0.244434,0.289764,0.478818,0.239752,0.239312,0.229589,0.243182,0.278581
4,BUN,0.234579,0.234759,0.275538,0.0,0.243482,0.279206,0.214579,0.193529,0.286102,0.244434,0.289764,0.478818,0.239752,0.239312,0.229589,0.243182,0.278581
5,Bilirubin,0.234579,0.234759,0.275538,0.0,0.243482,0.279206,0.214579,0.193529,0.286102,0.244434,0.289764,0.478818,0.239752,0.239312,0.229589,0.243182,0.278581
6,Cholesterol,0.234579,0.234759,0.275538,0.0,0.243482,0.279206,0.214579,0.193529,0.286102,0.244434,0.289764,0.478818,0.239752,0.239312,0.229589,0.243182,0.278581
7,Creatinine,0.234579,0.234759,0.275538,0.0,0.243482,0.279206,0.214579,0.193529,0.286102,0.244434,0.289764,0.478818,0.239752,0.239312,0.229589,0.243182,0.278581
8,DiasABP,0.234579,0.234759,0.275538,0.0,0.243482,0.279206,0.214579,0.193529,0.286102,0.244434,0.289764,0.478818,0.239752,0.239312,0.229589,0.243182,0.278581
9,FiO2,0.234579,0.234759,0.275538,0.0,0.243482,0.279206,0.214579,0.193529,0.286102,0.244434,0.289764,0.478818,0.239752,0.239312,0.229589,0.243182,0.278581


In [52]:
print("BRITS - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_brits_append_subgroups[i][j])

BRITS - MAE
************
General
-------------
ALP : 0.266644733489862
ALT : 0.23900266150625304
AST : 0.2854938721784185
Albumin : 0.3203138448779601
BUN : 0.24102697640531243
Bilirubin : 0.24960542526114163
Cholesterol : 0.2519937112900071
Creatinine : 0.2680821614127534
DiasABP : 0.246698476118878
FiO2 : 0.24946330367166566
GCS : 0.6473109930776697
Glucose : 0.28087676441624365
HCO3 : 0.2583291079764593
HCT : 0.24207674792433023
HR : 0.2410863112357303
K : 0.24739804172497776
Lactate : 0.25947332814140556
MAP : 0.2684142254010802
MechVent : 0.2627332565047349
Mg : 0.269234288659691
NIDiasABP : 0.2316112509677321
NIMAP : 0.24172368575241565
NISysABP : 0.2554514344344389
Na : 0.2665953580476764
PaCO2 : 0.2399055118050039
PaO2 : 0.25675305111767854
Platelets : 0.24342133381562436
RespRate : 0.2554206545499794
SaO2 : 0.26948549719958176
SysABP : 0.25779013358763925
Temp : 0.26511418636617634
TroponinI : 0.24506253875316777
TroponinT : 0.24178232451883985
Urine : 0.26819732476676317
WBC 